In [1]:
# Importing Libraries

import numpy as np  # np.inf is used to define the capacity of the infinite pitcher
import copy         # Used to copy only the values of the previous state. (Prevents pass by reference)

# Global Variables Initialization

CAPACITY = []   # Stores the maximum capacities of the jugs
visited = []    # List to store the explored states
priority = []   # Fringe that stores the un-explored states
target = 0      # Stores the target amount to be present in the infinite jug
start = []      # Stores the start state based on the input file
startState = "" # Stores the string representation of the start state

# Function to read input from text files

def fileReader(textfile):
    with open(textfile) as f:
        lines = f.readlines()

    Input = lines[0].strip('\n').split(',')
    CAPACITY.clear()
    visited.clear()
    priority.clear()

    for i in range(len(Input)):
        CAPACITY.append(int(Input[i]))
    CAPACITY.append(np.inf)

    global target 
    target = int(lines[1])
    global start 
    start = [0]*len(CAPACITY)
    global startState 
    startState = " " + str(start) + '\n'

####################################################################################################################

# Sorts the fringe based on the heuristic value

def prioritySort():
    priority.sort(key = lambda x: x.heuristic)

# # Returns the heuristic value for every node
# # h(n) = |target - (current amount of water in infinite pitcher + max(water in other pitchers ))|

def heuristic(state):
    
    tempState = state.pitchers.pop()
    infiniteCapacity = CAPACITY.pop()
    
    # if the current amount of water in pitcher is greater than, target + sum(capacities of all pitchers)
    if( tempState > target + sum(CAPACITY)):
        priority.clear()
    else:
        
        # if the remaining amount of water required to reach goal state exists, its heuristic becomes -1 ( Higher Priority)
        if( (target - tempState) in state.pitchers):
            state.heuristic = -1
        else:
            # # h(n) = |target - (current amount of water in infinite pitcher + max(water in other pitchers ))|
            state.heuristic = abs(target - ( tempState + max(state.pitchers)))
           
        # If in a level, if filling, transfering and emptying have the same heuristic. Transfering is given higher priority.
        if(state.parent == 't'):
            state.heuristic += 0.1
        elif state.parent == 'f':
            state.heuristic += 0.3
        elif state.parent == 'e':
            state.heuristic+= 0.5
        
    state.pitchers.append(tempState)
    CAPACITY.append(infiniteCapacity)
    
#########################################################################################################################    

# Class that defines the attributes of each state

class Pitcher:
    
    heuristic = 0  # Stores the calculated heuristic value for a state
    distance = 0   # Stores the distance of the current state from the start state
    path=[]        # Stores the path from the start state to the current state 
    parent = ''    # Stores the action of the parent state to reach the current state
                            # Transfer() --> t
                            # Fill()     --> f
                            # Empty()    --> e
            
    # Constructor to initialize the states
    def __init__(self, array, distance, path, parent):
        self.pitchers = array
        self.distance = distance
        self.path = path
        self.parent = parent
        # Constructor calls the heuristic function after state creation
        heuristic(self)
        
    # Fills the indexed pitcher from the infinite water source
    def fill(self, index):
        
        # Copy the previous state and create a new state
        temp = copy.deepcopy(self.pitchers)
        temp[index] = CAPACITY[index]

        priority.append(Pitcher(temp, self.distance + 1, self.path + " " + str(temp) + "\n", 'f'))
        prioritySort()
    
    # Empties the indexed pitcher to the ground
    def empty(self, index):
        
        temp = copy.deepcopy(self.pitchers)
        temp[index] = 0
        priority.append(Pitcher(temp, self.distance + 1, self.path + " " + str(temp) + "\n", 'e'))
        prioritySort()
    
    # Transfers water between pitchers
    def transfer(self, currIndex, transferIndex):
        
        temp = copy.deepcopy(self.pitchers)
        
        # If the transfer is to a Finite Pitcher
        if(transferIndex != len(temp) - 1):
              
            # If the remaining capacity at the destination is greater than the source
            if(CAPACITY[transferIndex] - temp[transferIndex]> temp[currIndex]):
                    
                temp[transferIndex] += temp[currIndex]
                temp[currIndex] = 0
            
            # If the remaining capacity at the destination is lesser than or equal the source
            elif (CAPACITY[transferIndex] - temp[transferIndex] <= temp[currIndex]):
                 
                temp[currIndex] = temp[currIndex] - (CAPACITY[transferIndex] - temp[transferIndex])    
                temp[transferIndex] = CAPACITY[transferIndex]
                
        
        #If the transfer is to an Infinite Pitcher
        else:
            
            temp[transferIndex] += temp[currIndex]
            temp[currIndex] = 0
            
        priority.append(Pitcher(temp, self.distance + 1, self.path + " " + str(temp) + "\n", 't'))
        prioritySort()
                
    #Prints all the remaining quantities of water in the pitcher
    def printPitcher(self):
        print(self.pitchers, self.heuristic, "Distance", self.distance)   
      
        
    # Successor function that generates the unvisited states and stores them in the fringe
    def successorFn(self):
        temp = self.pitchers
          
        # Finite Pitchers
        for curr in range(len(temp) - 1):
            
            if (temp[curr] == CAPACITY[curr]):
                
                # Transfering Logic --> Check for every other pithcher
                for dest in range(len(temp)):
                    if(dest != curr and temp[dest] < CAPACITY[dest]):
                        self.transfer(curr,dest) 
                
                self.empty(curr)
                  
            elif temp[curr] < CAPACITY[curr]:
                # Transfering Code
                for dest in range(len(temp)):
                    if(dest != curr and temp[dest] < CAPACITY[dest]):
                        self.transfer(curr,dest)
                self.fill(curr)
                self.empty(curr)
                
            elif temp[curr] == 0: 
                self.fill(curr) 
                
                
        # Infinite pitcher
        curr = len(temp) - 1
        if(temp[curr] != 0 ):
            for dest in range(len(temp) - 1):
                    if(temp[dest] < CAPACITY[dest]):
                        self.transfer(curr,dest)
            self.empty(curr)
            
                        
###########################################################################################################################

# Algorithm to perform graphsearch on the state space to reach goal state

def graphsearch():
    
    # Start State
    priority.append(Pitcher(start,0,startState,''))
    
    while(True):
        
        # If there are no more states to explore return -1
        if(len(priority) == 0):
            print("No Solution\n")
            return -1
            break
        
        # Explore most recent state
        curr = priority.pop(0)
        
        if(not curr.pitchers in visited):
            # Add current state as visited
            visited.append(curr.pitchers)
            
            #Goal Test
            if(curr.pitchers[len(curr.pitchers) - 1] == target):
                print(curr.path)
                print("Goal Node can be reached in", curr.distance,"steps.\n")
                return curr.distance
                break
            else:

                # Creation of next states from current state
                curr.successorFn() 
                        

# Unit Testing

In [2]:
# Capacity: 2,5,6,72 Target : 143
def test_input1():
    fileReader('input1.txt')
    assert graphsearch() == 7, "Should be 7"

# Capacity: 3,6 Target : 2
def test_input2():
    fileReader('input2.txt')
    assert graphsearch() == -1, "Should be -1"
    
# Capacity: 2 Target : 143
def test_input3():
    fileReader('input3.txt')
    assert graphsearch() == -1, "Should be -1"

# Capacity: 2,3,5,19,121,852  Target : 11443
def test_input4():
    fileReader('input4.txt')
    assert graphsearch() == 37, "Should be 37"
    
# Capacity: 3,5  Target : 1
def test_input5():
    fileReader('input5.txt')
    assert graphsearch() == 5, "Should be 5"

In [3]:
if __name__ == "__main__":
    test_input1()
    test_input2()
    test_input3()
    test_input4()
    test_input5()
    print("Everything Passed")

 [0, 0, 0, 0, 0]
 [0, 0, 0, 72, 0]
 [0, 0, 0, 0, 72]
 [0, 0, 0, 72, 72]
 [0, 0, 0, 0, 144]
 [0, 0, 6, 0, 138]
 [0, 5, 1, 0, 138]
 [0, 0, 1, 0, 143]

Goal Node can be reached in 7 steps.

No Solution

No Solution

 [0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 852, 0]
 [0, 0, 0, 0, 0, 0, 852]
 [0, 0, 0, 0, 0, 852, 852]
 [0, 0, 0, 0, 0, 0, 1704]
 [0, 0, 0, 0, 0, 852, 1704]
 [0, 0, 0, 0, 0, 0, 2556]
 [0, 0, 0, 0, 0, 852, 2556]
 [0, 0, 0, 0, 0, 0, 3408]
 [0, 0, 0, 0, 0, 852, 3408]
 [0, 0, 0, 0, 0, 0, 4260]
 [0, 0, 0, 0, 0, 852, 4260]
 [0, 0, 0, 0, 0, 0, 5112]
 [0, 0, 0, 0, 0, 852, 5112]
 [0, 0, 0, 0, 0, 0, 5964]
 [0, 0, 0, 0, 0, 852, 5964]
 [0, 0, 0, 0, 0, 0, 6816]
 [0, 0, 0, 0, 0, 852, 6816]
 [0, 0, 0, 0, 0, 0, 7668]
 [0, 0, 0, 0, 0, 852, 7668]
 [0, 0, 0, 0, 0, 0, 8520]
 [0, 0, 0, 0, 0, 852, 8520]
 [0, 0, 0, 0, 0, 0, 9372]
 [0, 0, 0, 0, 0, 852, 9372]
 [0, 0, 0, 0, 0, 0, 10224]
 [0, 0, 0, 0, 0, 852, 10224]
 [0, 0, 0, 0, 0, 0, 11076]
 [0, 0, 0, 0, 121, 0, 11076]
 [0, 0, 0, 0, 0, 0, 11197]
 [0, 0, 